   # Implementação algoritmo Rede Neural Artificial

In [1]:
import numpy as np

In [46]:
class RNA:
    
    def __init__(self):
        self.parameters = {} # Dicionário para os parâmetros
        self.learning_rate = 0
        self.nums_iterations = 0
        
        
    # Função para Inicialização randômica dos parametros do modelo
    def inicializa_parametros(self,dims_camada_entrada):
        """
        Inicializa parametros aleatórios para os pesos.
        
        Arg:
            dims_camada_entrada (list): [total de variáveis, nº neuronio camada 1, 
                                         nº neuronio camada 2,
                                         ..., 
                                         nº neuronio camada n,
                                         ]
        Retorna: 
            dict: dicionário com os valores iniciais dos pesos para cada camada do neurônio.
            
        """
        # Comprimento das dimensões das camadas, quantidade de camadas da rede
        comp = len(dims_camada_entrada)
        for i in range(1, comp):
            # Inicialização da matriz de pesos
            self.parameters["W" + str(i)] = np.random.randn(dims_camada_entrada[i], dims_camada_entrada[i - 1]) * 0.01
            # Inicialização do bias
            self.parameters["b" + str(i)] = np.zeros((dims_camada_entrada[i], 1))
        return self.parameters
    
    
    # Função sigmóide: utilizada para ativação da ultima camada
    @staticmethod
    def sigmoid(Z):
        """
        Converte qualquer valor real de Z para valores no intervalo de 0 e 1.
        
        Args:
            Z (float): qualquer número pertencente ao conjunto dos nº reais.
        
        Retorna:
            tupla : contendo o valor transformado pela função e o próprio valor de entrada
        """
        A = 1 / (1 + np.exp(-Z))
        return A, Z
    
    
    # Função de ativação ReLu (Rectified Linear Unit): Ativação camadas intermediárias
    @staticmethod
    def relu(Z):
        """
        Converte qualquer valor real de Z para 0 quando Z menor que 0,
        e retorna a imagem de Z para valores maiores que 0. 
        
        Args:
            Z (float): qualquer número pertencete ao conjunto dos nº reais
        
        Retorna:
            tupla: 
        """
        A = abs(Z * (Z > 0))
        return A, Z
    
    
    # Ativação linear
    def linear_activation(A, W, b):
        """
        Realiza a operação de ativação por camada
        
        Args: 
            A (numpy.ndarray): matriz com os dados de entrada da camada
            W (numpy.ndarray): matriz de pesos da camada
            b (numpy.ndarray): bias
        
        Retorna:
            tupla: contendo o valor de ativação e das memórias das matrizes dos pesos. 
        """
        Z = np.dot(W, A) + b
        cache = (A, W, b)
        return Z, cache
    
    
    # Forward Propagation
    # Movimento para frente (forward)
    def forward(A_prev, W, b, activation):
        """
        Aplica a função de ativação nos resultados da multiplcaçãdo dos dados pelos coeficientes
        
        Args: 
            A_prev (numpy.ndarray): Dados entrada prévios;
            W (numpy.ndarray): Matriz de coeficientes da camada;
            b (numpy.ndarray): bias da camada;
            activation (str): tipo da função de ativação "sigmoid" ou "relu"
        
        Retorna:
            tupla: A: os resultados dos valores passado pela função de ativação;
                   cache: mémoria dos dados iniciais, matriz de coeficiente e bias
        """

        # Se a função de ativação for Sigmoid, entramos neste bloco
        if activation == "sigmoid":
            Z, linear_cache = linear_activation(A_prev, W, b)
            A, activation_cache = sigmoid(Z)

        # Se não, se for ReLu, entramos neste bloco    
        elif activation == "relu":
            Z, linear_cache = linear_activation(A_prev, W, b)
            A, activation_cache = relu(Z)

        cache = (linear_cache, activation_cache)

        return A, cache

    
    # Propagação para frente
    def forward_propagation(X, parameters):
        """
        Realiza as multiplicações de matrizes de cada camada da rede.
        
        Args:
            X (numpy.ndarray): Valores dos dados de treinamento;
            parameters (dict): dicionario das matrizes de coeficientes e bias
        
        Retorna:
            tupla: A_last -> valores de saida da última camada;
                            caches -> lista com os valores calculados nas camadas internas;
        """
        # Lista de valores anteriores (cache)
        caches = []
        # Dados de entrada
        A = X
        # Comprimento dos parâmetros
        L = len(parameters) // 2
        # Loop
        for i in range(1, L):
            # Guarda o valor prévio de A
            A_prev = A
            # Executa o forward
            A, cache = forward(A_prev, parameters["W" + str(i)], parameters["b" + str(i)], activation = "relu")
            # Grava o cache
            caches.append(cache)
        # Saída na última camada
        A_last, cache = forward(A, parameters["W" + str(L)], parameters["b" + str(L)], activation = "sigmoid")
        # Grava o cache
        caches.append(cache)
        return(A_last, caches)
    
    
    # Função de custo (ou função de erro)
    def calcula_custo(A_last, Y):
        """
        Calcula a função custo dado a comparação de valores previstos e valores de treinamento reais
        
        Args: 
            A_last (numpy.ndarray): Matriz da última camada contendo as estimativas do modelo.
            Y (numpy.ndarray): valores reais dos dados de treinamento
        
        Retorna: 
            numpy.ndarray: valor da função custo
        """
        # Ajusta o shape de Y para obter seu comprimento (total de elementos)
        m = Y.shape[1]
        # Calcula o custo comparando valor real e previso (função logit para custo)
        custo = (-1 / m) * np.sum((Y * np.log(A_last)) + ((1 - Y) * np.log(1 - A_last)))
        # Ajusta o shape do custo
        custo = np.squeeze(custo)
        return(custo)
    

    ## == BackPropagation == ##
    # Função sigmoid para o backpropagation ( Derivada da função sigmoid)
    def sigmoid_backward(da, Z):
        """
        Calcula a derivada da função de ativação sigmoid, sua taxa de variação.
        
        Args:
            da (float): derivada da previsão final da rede (feita ao final do Forward Propagation).
            Z (float): valores da multiplicação dos pesos pelos dados da camada.
        
        Retorna: 
            float: taxa de variação da função sigmoid.
        """
        # Calculamos a derivada de Z
        dg = (1 / (1 + np.exp(-Z))) * (1 - (1 / (1 + np.exp(-Z))))
        # Encontramos a mudança na derivada de z
        dz = da * dg # regra da cadeia*
        return dz
    
    
    # Função relu para o backpropagation 
    def relu_backward(da, Z):
        """
        Calcula a derivada da função de ativação relu
        
        Args: 
            da (float): derivada da previsão final da rede (feita ao final do Forward Propagation).
            Z (float): valores da multiplicação dos pesos pelos dados da camada.
            
        Retorna: 
            float: taxa de variação da função relu.
        """
        dg = 1 * ( Z >= 0)
        dz = da * dg # regra da cadeia*
        return dz
    
    
    # Ativação linear para o backpropagation
    def linear_backward_function(dz, cache):
        """
        Calcula as derivadas da operação backward propragation
        
        Args: 
            dz (float): taxa de variação da função de ativação
            cache (tupla): (linear_cache, activation_cache) 
                            linear_cache -> resultado da multiplicação dos pesos pelos dados da camada.
                            activation_cache -> resultado da função de ativação dado o linear_cache como entrada.
        
        Retorna: 
            tupla: derivada da operação (W*dz), derivada da matriz peso W (dz*A_prev), derivada do bias.
        
        """
        # Recebe os valores do cache (memória)
        A_prev, W, b = cache
        # Shape de m
        m = A_prev.shape[1]
        # Calcula a derivada de W (resultado da operação com dz)
        dW = (1 / m) * np.dot(dz, A_prev.T)
        # Calcula a derivada de b (resultado da operação com dz)
        db = (1 / m) * np.sum(dz, axis = 1, keepdims = True)
        # Calcula a derivada da operação
        dA_prev = np.dot(W.T, dz)
        return dA_prev, dW, db


    def linear_activation_backward(dA, cache, activation):
        """
        Define o tipo de ativação ( relu ou sigmoid)
        
        Args:
            dA: derivada da previsão final da rede (feita ao final do Forward Propagation)
            cache: cache mémoria dos dados iniciais, matriz de coeficiente e bias
            activation: Define a função de ativação "relu" ou "sigmoid"
        
        Retorna:
            tupla: derivada da operação (W*dz), derivada da matriz peso W (dz*A_prev), derivada do bias.
        """
        # Extrai o cache
        linear_cache, activation_cache = cache
        # Verifica se a ativação é relu
        if activation == "relu":
            dZ = relu_backward(dA, activation_cache)
            dA_prev, dW, db = linear_backward_function(dZ, linear_cache)
        # Verifica se a ativação é sigmoid
        if activation == "sigmoid":
            dZ = sigmoid_backward(dA, activation_cache)
            dA_prev, dW, db = linear_backward_function(dZ, linear_cache)
        return dA_prev, dW, db


    def backward_propagation(AL, Y, caches):
        """
        Calcula os gradientes para atualização dos pesos
        
        Args:
            AL (numpy.ndarray): matriz das previsões realizadas pelo foward.
            Y (numpy.ndarray): matriz dos resultados reais.
            caches (list): lista com os valores calculados nas camadas internas;
        
        Retorna:
            dict: Dicionário contendo os gradientes dos dados, dos pesos e do bias.
        """
        # Dicionário para os gradientes
        grads = {}
        # Comprimento dos dados (que estão no cache)
        L = len(caches)
        # Extrai o comprimento para o valor de m
        m = AL.shape[1]
        # Ajusta o shape de Y
        Y = Y.reshape(AL.shape)
        # Calcula a derivada da previsão final da rede (feita ao final do Forward Propagation)
        dAL = -((Y / AL) - ((1 - Y) / (1 - AL)))
        # Captura o valor corrente do cache
        current_cache = caches[L - 1]
        # Gera a lista de gradiente para os dados, os pesos e o bias
        # Fazemos isso uma vez, pois estamos na parte final da rede, iniciando o caminho de volta
        grads["dA" + str(L - 1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
        # Loop para calcular a derivada durante as ativações lineares com a relu
        for l in reversed(range(L - 1)):
            # Cache atual
            current_cache = caches[l]
            # Calcula as derivadas
            dA_prev, dW, db = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "relu")
            # Alimenta os gradientes na lista, usando o índice respectivo
            grads["dA" + str(l)] = dA_prev
            grads["dW" + str(l + 1)] = dW
            grads["db" + str(l + 1)] = db        
        return grads

    
    # Função de atualização de pesos
    def atualiza_pesos(parameters, grads, learning_rate):
        """
        Realiza a atualização dos pesos em cada camada da rede.
        
        Args:
            parameters (dict): dicionário contendo todas matrizes de pesos.
            grads (dict): dicionário contendo todas as matrizes dos gradientes calculados. 
            learning_rate (float): taxa de aprendizado.
        
        Retorna:
            dict: dicionário contendo as matrizes de pesos atualizados pelo gradiente do backpropagation.
        
        """
        # Comprimento da estrutura de dados com os parâmetros (pesos e bias)
        L = len(parameters)//2
        # Loop para atualização dos pesos
        for l in range(L):
            # Atualização dos pesos
            parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - (learning_rate * grads["dW" + str(l + 1)])
            # Atualização do bias
            parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - (learning_rate * grads["db" + str(l + 1)])
        return parameters
    
    
    # Modelo completo da rede neural
    def modeloNN(X, Y, dims_camada_entrada, learning_rate = 0.0075, num_iterations = 100):
        """
        Executa o treinamento do modelo da Rede Neural Artificial.
        
        Args:
            X (numpy.ndarray): Dados treinamento variáveis independentes.
            Y (numpy.ndarray): Dados treinamento variáveis dependentes.
            dims_camada_entrada: Dimensões das camadas e estruturas de Neurônios.
            learning_rate (float): Taxa de apresendizado do modelo.
            num_iterations (int): Quantidade de iterações para o treinamento do modelo.
            
        Retorna: 
            tupla: dicionário dos parametros treinados, lista dos custos por iteração de treinamento. 
        """
        
        # Lista para receber o custo a cada época de treinamento
        custos = []

        # Inicializa os parâmetros
        parametros = inicializa_parametros(dims_camada_entrada)

        # Loop pelo número de iterações (épocas)
        for i in range(num_iterations):

            # Etapa Forward Propagation
            AL, caches = forward_propagation(X, parametros)

            # Calcula o custo
            custo = calcula_custo(AL, Y)

            # Etapa Backward Propagation (Atualização dos pesos)
            # Nota: ao invés de AL e Y, poderíamos passar somente o valor do custo
            # Estamos passando o valor de AL e Y para fique claro didaticamente o que está sendo feito
            gradientes = backward_propagation(AL, Y, caches)

            # Atualiza os pesos
            parametros = atualiza_pesos(parametros, gradientes, learning_rate)

            # Print do valor intermediário do custo
            # A redução do custo indica o aprendizado do modelo
            if i % 10 == 0:
                print("Custo Após " + str(i) + " iterações é " + str(custo))
                custos.append(custo)

        return parametros, custos
    
    
    # Função para fazer as previsões
    # Não precisamos do Backpropagation pois ao fazer previsões como o modelo treinado, 
    # teremos os melhores valores de pesos (parametros)
    def predict(X, parametros):
        AL, caches = forward_propagation(X, parametros)
        return AL
    

In [54]:
obj1 = RNA()
obj1.inicializa_parametros([30,50,20,5,1])
obj1.parameters

{'W1': array([[-5.63630455e-04, -9.90960229e-03, -6.92388239e-03, ...,
          2.13546958e-03, -5.36667137e-03,  3.82150714e-03],
        [-1.13175880e-04, -1.28247133e-02,  4.48982770e-03, ...,
          1.66075648e-02, -1.13295415e-02,  3.76920882e-03],
        [ 7.88615062e-03,  1.49771723e-02,  1.47014008e-02, ...,
         -1.80061007e-03, -9.76798391e-03, -8.62071135e-03],
        ...,
        [-9.33836051e-03, -1.13140559e-02, -4.98414580e-03, ...,
          3.98528031e-03,  5.90763699e-04,  3.29522816e-03],
        [-5.91532148e-03,  2.09890397e-02, -2.52679389e-03, ...,
          8.89972579e-03, -9.50421771e-03,  2.09793011e-02],
        [-1.35372064e-03, -9.41887958e-05, -1.50589675e-02, ...,
         -1.03862118e-03,  3.47324125e-03, -1.08791655e-02]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
       

In [55]:
parametros = obj1.parameters
parametros['W1'].shape

(50, 30)

In [12]:
list(range(1,3))

[1, 2]

In [56]:
len(parametros)

8

In [53]:
len({'1':1, '2':2, '3':3})

3